In [8]:
import cv2
from ultralytics import YOLO
import datetime

# تحميل نموذج YOLO
model = YOLO("yolov8n.pt")

# فتح الفيديو
cap = cv2.VideoCapture("input.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# إعداد الفيديو الناتج
out = cv2.VideoWriter("Offside_Result1.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# إعدادات الوقت
match_start_minute = 22  
frame_num = 0
offside_frame = None
offside_time = ""
offside_detected = False
last_defender_x = None
attacker_x = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)[0]
    players = []
    ball = None

    for box in results.boxes:
        cls_id = int(box.cls[0].item())
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        if cls_id == 0:
            players.append((center_x, center_y))
        elif cls_id == 32:
            ball = (center_x, center_y)

    if ball and len(players) >= 3:
        players_sorted = sorted(players, key=lambda p: p[0])
        last_defender_x = players_sorted[-2][0]
        attacker_x = players_sorted[-1][0]

        if attacker_x > last_defender_x + 20 and not offside_detected:
            offside_detected = True
            offside_frame = frame_num

            # حساب الدقيقة الفعلية للمباراة
            match_minute = match_start_minute + int(frame_num / fps / 60)
            offside_time = f"{match_minute} min"

            print(f"Offside detected at: {offside_time}")

    # عرض البيانات فقط وقت التسلل ولمدة 5 ثوانٍ
    if offside_detected and frame_num - offside_frame < fps * 5:
        cv2.line(frame, (last_defender_x, 0), (last_defender_x, frame_height), (0, 0, 255), 5)
        cv2.putText(frame, "OFFSIDE", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
        cv2.putText(frame, f"Offside at minute: {offside_time}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

    out.write(frame)
    frame_num += 1

cap.release()
out.release()
print("تم حفظ الفيديو الناتج باسم: Offside_Result1.mp4")


Offside detected at: 22 min
تم حفظ الفيديو الناتج باسم: Offside_Result1.mp4


In [13]:
import cv2
import torch
import numpy as np

# تحميل نموذج YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# فتح الفيديو
video = cv2.VideoCapture('input1.mp4')

# إعداد الكتابة إلى ملف فيديو بصيغة MP4 باستخدام الترميز H264
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # الترميز H264
output_video = cv2.VideoWriter('offside_result2.mp4', fourcc, 30, (int(video.get(3)), int(video.get(4))))

# متغير لتخزين حالة التسلل
offside_warning = False

# حساب لحظة تمرير الكرة
pass_frame = None

while True:
    ret, frame = video.read()
    if not ret:
        break

    # تطبيق النموذج على الإطار
    results = model(frame)

    # استخراج معلومات اللاعبين والكرة
    detections = results.pandas().xyxy[0]
    
    players = detections[detections['name'] == 'person']
    ball = detections[detections['name'] == 'sports ball']

    # رسم الكائنات
    for _, row in players.iterrows():
        x1, y1, x2, y2 = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

    for _, row in ball.iterrows():
        x1, y1, x2, y2 = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
        ball_center = ((x1+x2)//2, (y1+y2)//2)
        cv2.circle(frame, ball_center, 10, (0,0,255), -1)

        # تحديد لحظة تمرير الكرة (مبدئيًا نعتبر أن التمرير حدث عندما تكون الكرة في الهواء)
        if pass_frame is None and y1 < frame.shape[0] // 2:
            pass_frame = video.get(cv2.CAP_PROP_POS_FRAMES)
    
    if pass_frame:
        # مقارنة المواقع للتأكد من التسلل
        player_positions = []
        for _, row in players.iterrows():
            player_center = ((row['xmin'] + row['xmax']) // 2, (row['ymin'] + row['ymax']) // 2)
            player_positions.append(player_center)
        
        # فرضية بسيطة: المهاجم هو الأقرب إلى المرمى (أعلى في الصورة)
        attackers = [pos for pos in player_positions if pos[1] < frame.shape[0] // 2]  # المهاجمين هم الأعلى في الصورة
        defenders = [pos for pos in player_positions if pos[1] > frame.shape[0] // 2]  # المدافعين هم الأسفل في الصورة
        
        if attackers and defenders:
            # تحديد آخر مدافع
            last_defender = min(defenders, key=lambda x: x[1])  # المدافع الأخير هو الأسفل في الصورة
            for attacker in attackers:
                if attacker[1] < last_defender[1]:  # المهاجم في حالة تسلل إذا كان أعلى من المدافع الأخير
                    offside_warning = True
                    cv2.putText(frame, "Offside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # إضافة الإطار المعدل إلى الفيديو
    output_video.write(frame)

    # عرض الإطار
    cv2.imshow("Offside Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# إنهاء الفيديو
video.release()
output_video.release()
cv2.destroyAllWindows()

C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\ACER/.cache\torch\hub\master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.2', 'setuptools>=70.0.0'] not found, attempting AutoUpdate...
   ---------------------------------------- 2.7/2.7 MB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 1.3/1.3 MB 3.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.2
    Uninstalling setuptools-69.0.2:
      Successfully uninstalled setuptools-69.0.2
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 10.1.0
    Uninstalling Pillow-10.1.0:
      Successfully uninstalled Pillow-10.1.0

requirements: AutoUpdate success  37.8s, installed 4 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.2', 'setuptools>=70.0.0']
requirements:  Restart runtime or rerun command for upda

YOLOv5  2025-4-17 Python-3.12.1 torch-2.6.0+cpu CPU

100%|█████████████████████████████████████████████████████████████████████████████| 14.1M/14.1M [00:05<00:00, 2.93MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\ACER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ACER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ACER/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\ACER/

In [21]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل النموذج المدرب
model = model = YOLO('C:/Users/ACER/Desktop/Jupyter/yolov8x.pt') # استبدل بالمسار الصحيح لنموذجك

# تحميل الفيديو
input_path = 'input1.mp4'
cap = cv2.VideoCapture(input_path)

# إعداد الفيديو الناتج
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_offside2.mp4', fourcc, fps, (w, h))

frame_idx = 0
ball_prev = None
pass_frame_idx = None
offside_player = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    boxes = results.boxes
    player_boxes = []
    ball_box = None

    for box in boxes:
        cls = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        
        if cls == 0:  # نفترض class 0 = لاعب
            player_boxes.append(((x1 + x2) // 2, (y1 + y2) // 2, x1, y1, x2, y2))
        elif cls == 32:  # نفترض class 1 = كرة
            ball_box = ((x1 + x2) // 2, (y1 + y2) // 2)

    # تقدير لحظة التمرير
    if ball_box and ball_prev:
        dx = ball_box[0] - ball_prev[0]
        dy = ball_box[1] - ball_prev[1]
        speed = np.sqrt(dx**2 + dy**2)
        if speed > 15 and pass_frame_idx is None:  # السرعة تدل على تمريرة
            pass_frame_idx = frame_idx
            print(f"تمرير عند الفريم {frame_idx}")
            # نحدد اللاعب الذي استلم الكرة لاحقاً
            predicted_receiver_pos = ball_box[0]  # إحداثي X فقط لتبسيط الفكرة
            player_boxes_sorted = sorted(player_boxes, key=lambda p: abs(p[0] - predicted_receiver_pos))
            potential_receiver = player_boxes_sorted[0]
            offside_player = potential_receiver

    ball_prev = ball_box

    # رسم الإطار على اللاعب المتسلل
    if pass_frame_idx is not None and offside_player:
        cx, cy, x1, y1, x2, y2 = offside_player
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
        cv2.putText(frame, 'Offside?', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()


0: 384x640 21 persons, 2 clocks, 2573.0ms
Speed: 72.3ms preprocess, 2573.0ms inference, 45.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 2 clocks, 2387.7ms
Speed: 9.3ms preprocess, 2387.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2315.5ms
Speed: 17.9ms preprocess, 2315.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2239.2ms
Speed: 6.9ms preprocess, 2239.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 clock, 2147.0ms
Speed: 7.4ms preprocess, 2147.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2198.8ms
Speed: 6.4ms preprocess, 2198.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 clock, 2729.7ms
Speed: 5.5ms preprocess, 2729.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

In [20]:
import cv2
from ultralytics import YOLO

# تحميل النموذج
model = YOLO('C:/Users/ACER/Desktop/Jupyter/yolov8x.pt')

# تحميل الفيديو
cap = cv2.VideoCapture('input1.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    boxes = results.boxes

    for box in boxes:
        cls = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        label = f"Class {cls}"

        # رسم المستطيل والنص
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Class Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 21 persons, 2 clocks, 3334.5ms
Speed: 91.5ms preprocess, 3334.5ms inference, 56.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 2 clocks, 2645.5ms
Speed: 33.3ms preprocess, 2645.5ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2489.2ms
Speed: 17.1ms preprocess, 2489.2ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2391.6ms
Speed: 15.3ms preprocess, 2391.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 clock, 2357.3ms
Speed: 7.8ms preprocess, 2357.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2754.3ms
Speed: 11.3ms preprocess, 2754.3ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 clock, 2701.9ms
Speed: 9.4ms preprocess, 2701.9ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 3

In [23]:
import cv2
from ultralytics import YOLO
import numpy as np

model = YOLO('C:/Users/ACER/Desktop/Jupyter/yolov8x.pt')
cap = cv2.VideoCapture('input1.mp4')
output_path = 'output_offside2.mp4'

fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

pass_frame_idx = None
offside_player = None

frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    boxes = results.boxes

    player_boxes = []
    ball_box = None

    for box in boxes:
        cls = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # تعديل الكلاسات:
        if cls == 0:  # player
            player_boxes.append((x1, y1, x2, y2))
        elif cls == 32:  # ball
            ball_box = (x1, y1, x2, y2)

    # تحديد لحظة التمرير
    if ball_box and len(player_boxes) > 0:
        bx1, by1, bx2, by2 = ball_box
        ball_center = ((bx1 + bx2) // 2, (by1 + by2) // 2)

        if frame_idx > 0 and last_ball_center:
            dx = ball_center[0] - last_ball_center[0]
            dy = ball_center[1] - last_ball_center[1]
            speed = np.sqrt(dx**2 + dy**2)

            if speed > 5 and pass_frame_idx is None:
                pass_frame_idx = frame_idx
                print(f"Pass detected at frame {frame_idx}")

        last_ball_center = ball_center
    else:
        last_ball_center = None

    # تحديد اللاعب المتسلل
    if pass_frame_idx and frame_idx == pass_frame_idx + 1:
        if ball_box and len(player_boxes) > 0:
            bx1, by1, bx2, by2 = ball_box
            ball_x = (bx1 + bx2) // 2

            max_x = max((x2 for (x1, y1, x2, y2) in player_boxes))
            for (x1, y1, x2, y2) in player_boxes:
                player_center_x = (x1 + x2) // 2
                if player_center_x > max_x - 10:  # تعديل لحساسية الكشف
                    offside_player = (x1, y1, x2, y2)
                    print(f"Offside player at frame {frame_idx}: {offside_player}")
                    break

    # رسم العناصر
    for (x1, y1, x2, y2) in player_boxes:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    if ball_box:
        bx1, by1, bx2, by2 = ball_box
        cv2.rectangle(frame, (bx1, by1), (bx2, by2), (255, 255, 0), 2)

    if offside_player:
        ox1, oy1, ox2, oy2 = offside_player
        cv2.rectangle(frame, (ox1, oy1), (ox2, oy2), (0, 0, 255), 3)
        cv2.putText(frame, "Offside?", (ox1, oy1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()


0: 384x640 21 persons, 2 clocks, 2766.6ms
Speed: 69.1ms preprocess, 2766.6ms inference, 39.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 2 clocks, 2145.2ms
Speed: 6.8ms preprocess, 2145.2ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2431.2ms
Speed: 6.8ms preprocess, 2431.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2147.8ms
Speed: 9.7ms preprocess, 2147.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 clock, 2142.4ms
Speed: 12.4ms preprocess, 2142.4ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2151.8ms
Speed: 9.2ms preprocess, 2151.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 clock, 2117.6ms
Speed: 6.2ms preprocess, 2117.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

In [24]:
import cv2
from ultralytics import YOLO
import numpy as np

model = YOLO('C:/Users/ACER/Desktop/Jupyter/yolov8x.pt')
cap = cv2.VideoCapture('input1.mp4')
output_path = 'output_offside_only.mp4'

fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

pass_frame_idx = None
offside_player = None
last_ball_center = None
frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    boxes = results.boxes

    player_boxes = []
    ball_box = None

    for box in boxes:
        cls = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        if cls == 0:
            player_boxes.append((x1, y1, x2, y2))
        elif cls == 32:
            ball_box = (x1, y1, x2, y2)

    if ball_box and len(player_boxes) > 0:
        bx1, by1, bx2, by2 = ball_box
        ball_center = ((bx1 + bx2) // 2, (by1 + by2) // 2)

        if last_ball_center:
            dx = ball_center[0] - last_ball_center[0]
            dy = ball_center[1] - last_ball_center[1]
            speed = np.sqrt(dx**2 + dy**2)

            if speed > 5 and pass_frame_idx is None:
                pass_frame_idx = frame_idx
                print(f"Pass detected at frame {frame_idx}")

        last_ball_center = ball_center
    else:
        last_ball_center = None

    # في الفريم بعد التمريرة نحدد المتسلل
    if pass_frame_idx and frame_idx == pass_frame_idx + 1 and not offside_player:
        if len(player_boxes) > 0:
            # نفترض أن الفريق يهاجم لليمين، فالمتسلل هو اللاعب الأكثر بعداً في x
            offside_player = max(player_boxes, key=lambda box: (box[0] + box[2]) // 2)
            print(f"Offside player at frame {frame_idx}: {offside_player}")

    # فقط رسم اللاعب المتسلل إن تم كشفه
    if offside_player:
        x1, y1, x2, y2 = offside_player
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
        cv2.putText(frame, "Offside?", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()


0: 384x640 21 persons, 2 clocks, 2679.9ms
Speed: 7.8ms preprocess, 2679.9ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 2 clocks, 2396.9ms
Speed: 12.8ms preprocess, 2396.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2106.9ms
Speed: 5.3ms preprocess, 2106.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2439.8ms
Speed: 9.8ms preprocess, 2439.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 clock, 2101.6ms
Speed: 6.4ms preprocess, 2101.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 clock, 2139.9ms
Speed: 7.1ms preprocess, 2139.9ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 clock, 2090.9ms
Speed: 11.0ms preprocess, 2090.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6

In [30]:
import cv2
from ultralytics import YOLO
import pytesseract

# تحميل نموذج YOLO للكشف
model = YOLO("yolov8n.pt")

# فتح الفيديو
video_path = "match_foul_clip.mp4"
cap = cv2.VideoCapture(video_path)

# OCR لإعداد تيسراكت
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # عدّل المسار حسب جهازك

# إعداد الفيديو للحفظ
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("foul_auto_detected.mp4", fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    results = model(frame)
    annotated_frame = results[0].plot()

    # محاولة قراءة أرقام على ظهر اللاعبين (OCR)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ocr_result = pytesseract.image_to_string(gray, config='--psm 6')
    found_number = ''.join(filter(str.isdigit, ocr_result))

    # محاكاة كشف الفاول في الإطار رقم 150-160
    if 150 <= frame_count <= 160:
        foul_type = "تدخل قوي"
        card_color = "بطاقة صفراء" if "تدخل" in foul_type else "بطاقة حمراء"

        cv2.putText(annotated_frame, f"لاعب رقم: {found_number if found_number else 'غير معروف'}",
                    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(annotated_frame, f"الخطأ: {foul_type}",
                    (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(annotated_frame, card_color,
                    (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2,
                    (0, 255, 255) if "صفراء" in card_color else (0, 0, 255), 3)

    out.write(annotated_frame)
    cv2.imshow("Auto Foul Detection", annotated_frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 12 persons, 883.4ms
Speed: 478.3ms preprocess, 883.4ms inference, 68.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 1128.3ms
Speed: 7.1ms preprocess, 1128.3ms inference, 7.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 persons, 222.2ms
Speed: 3.7ms preprocess, 222.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 186.2ms
Speed: 5.0ms preprocess, 186.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 283.4ms
Speed: 3.5ms preprocess, 283.4ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 242.6ms
Speed: 5.2ms preprocess, 242.6ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 212.3ms
Speed: 4.7ms preprocess, 212.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 241.4ms
Speed: 4.5ms preprocess, 241.4ms inference, 1.9ms pos

In [31]:
import cv2

cap = cv2.VideoCapture("match_foul_clip.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # نرسم نص تجريبي
    cv2.putText(frame, "هذا اختبار عرض النص", (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    cv2.imshow("Test Text", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [32]:
import cv2

# تأكد إن مسار الفيديو صحيح
cap = cv2.VideoCapture("match_foul_clip.mp4")

if not cap.isOpened():
    print("فشل في تحميل الفيديو، تأكد من اسم الملف أو المسار!")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("انتهى الفيديو أو فيه مشكلة في القراءة")
        break

    cv2.imshow("عرض الفيديو فقط", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

انتهى الفيديو أو فيه مشكلة في القراءة


In [33]:
import cv2
import os

video_path = "match_foul_clip.mp4"

# تأكد إذا الملف فعلاً موجود
if not os.path.exists(video_path):
    print("الفيديو غير موجود في هذا المسار:", video_path)
    exit()

# نحاول نفتح الفيديو
cap = cv2.VideoCapture(video_path)

# نتحقق إذا فتح بنجاح
if not cap.isOpened():
    print("تعذر فتح الفيديو! قد يكون نوعه غير مدعوم.")
    exit()

print("الفيديو يعمل! جاري العرض... اضغط Q للخروج.")

# نعرض كل إطار
while True:
    ret, frame = cap.read()
    if not ret:
        print("انتهى الفيديو.")
        break

    cv2.imshow("Video Test", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

الفيديو يعمل! جاري العرض... اضغط Q للخروج.
انتهى الفيديو.


In [34]:
import cv2

video_path = "match_foul_clip.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("ما قدر يفتح الفيديو.")
    exit()

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print(f"انتهى الفيديو عند الفريم رقم: {frame_count}")
        break

    frame_count += 1
    cv2.putText(frame, f"Frame: {frame_count}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Counting Frames", frame)
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

انتهى الفيديو عند الفريم رقم: 371


In [35]:
import cv2
from ultralytics import YOLO

# تحميل نموذج YOLO (تأكد أنك محمله أو استخدم yolov8n.pt)
model = YOLO("yolov8n.pt")

video_path = "match_foul_clip.mp4"
cap = cv2.VideoCapture(video_path)

# إعداد الفيديو للحفظ
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("foul_detected_output.mp4", fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print(f"انتهى الفيديو عند الفريم رقم: {frame_count}")
        break

    frame_count += 1
    results = model(frame)
    annotated_frame = results[0].plot()

    # نعرض فقط معلومات الفاول بين الفريم 150 إلى 160 كمثال
    if 150 <= frame_count <= 160:
        cv2.putText(annotated_frame, f"لاعب رقم: 10", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(annotated_frame, f"الخطأ: تدخل عنيف", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(annotated_frame, "بطاقة صفراء", (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

    # رقم الفريم يظهر دائمًا
    cv2.putText(annotated_frame, f"Frame: {frame_count}", (50, height - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    out.write(annotated_frame)
    cv2.imshow("Foul Detection", annotated_frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 12 persons, 508.6ms
Speed: 112.8ms preprocess, 508.6ms inference, 58.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 599.0ms
Speed: 15.4ms preprocess, 599.0ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 persons, 181.7ms
Speed: 5.4ms preprocess, 181.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 189.5ms
Speed: 4.7ms preprocess, 189.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 298.0ms
Speed: 8.6ms preprocess, 298.0ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 346.9ms
Speed: 6.5ms preprocess, 346.9ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 206.3ms
Speed: 6.3ms preprocess, 206.3ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 179.8ms
Speed: 6.4ms preprocess, 179.8ms inference, 1.3ms post

In [37]:
import sys
print(sys.executable)

C:\Users\ACER\AppData\Local\Programs\Python\Python312\python.exe


In [39]:
pip install easyocr

Note: you may need to restart the kernel to use updated packages.


In [41]:
!C:\Users\ACER\AppData\Local\Programs\Python\Python312\python.exe -m pip install easyocr

In [43]:
import cv2
from ultralytics import YOLO

# تحميل النموذج
model = YOLO("yolov8n.pt")

video_path = "match_foul_clip.mp4"
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("foul_output_english.mp4", fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print(f"Video ended at frame {frame_count}")
        break

    frame_count += 1
    results = model(frame)
    annotated_frame = results[0].plot()

    if 150 <= frame_count <= 160:
        cv2.putText(annotated_frame, "Player Number: 10", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(annotated_frame, "Foul: Dangerous Tackle", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(annotated_frame, "Yellow Card", (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

    cv2.putText(annotated_frame, f"Frame: {frame_count}", (50, height - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    out.write(annotated_frame)
    cv2.imshow("Foul Detection - English", annotated_frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 12 persons, 1102.4ms
Speed: 125.8ms preprocess, 1102.4ms inference, 66.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 270.0ms
Speed: 30.0ms preprocess, 270.0ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 persons, 222.7ms
Speed: 7.2ms preprocess, 222.7ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 197.0ms
Speed: 4.4ms preprocess, 197.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 175.5ms
Speed: 5.6ms preprocess, 175.5ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 191.6ms
Speed: 6.4ms preprocess, 191.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 177.7ms
Speed: 5.4ms preprocess, 177.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 160.3ms
Speed: 5.4ms preprocess, 160.3ms inference, 1.7ms po

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture("match_foul_clip.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("auto_foul_detection.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

prev_frame = None
frame_count = 0
foul_frame_detected = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # حساب الفرق بين الفريم الحالي والسابق
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if prev_frame is not None:
        frame_diff = cv2.absdiff(prev_frame, gray)
        motion_score = np.sum(frame_diff)

        # عتبة الحركة (كلما زادت، قد يدل على تصادم)
        if motion_score > 1_000_000:
            foul_frame_detected.append(frame_count)

    prev_frame = gray

    # تحليل اللاعبين
    results = model(frame)
    annotated = results[0].plot()

    # إذا الفريم تم تحديده كفريم فيه فاول
    if frame_count in foul_frame_detected:
        cv2.putText(annotated, "Player Number: 7", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(annotated, "Foul Detected", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(annotated, "Yellow Card", (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

    cv2.imshow("Auto Foul Detection", annotated)
    out.write(annotated)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()